In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, LSTM, Dropout, concatenate, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder

In [4]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Rescaling
from tensorflow.keras import Model
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam

TensorFlow version: 2.11.0


In [ ]:
data = pd.read_pickle('total_data_pickle')

In [57]:
model = tf.keras.models.load_model("model_parallel_azad.hdf5")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 298, 128)     512         ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 296, 64)      24640       ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 148, 64)      0           ['conv1d_1[0][0]']               
                                                                                              

In [58]:
model

In [ ]:
labels = data["label"].values

In [ ]:
values = data.drop(columns=['label'])

In [5]:
# from testdata import *
# from traindata import *


# sampleSize = 300
# overlapRatio = 10
# path_train = "df_f_feather"
# path_test = "test_df"

# train = trainData()
# test = testData()


# values, labels = train.prepareTrainData(sampleSize, overlapRatio, path_train)
# test_values, test_labels = test.prepareTestData(sampleSize, overlapRatio, path_test)

current_folder_path = os.getcwd()
parent_folder_path = os.path.dirname(current_folder_path)
data_save_path = os.path.join(parent_folder_path, 'data')

In [16]:
path = parent_folder_path + "\\train_data_values_300_10"
values= pd.read_pickle(path)
path = parent_folder_path + "\\train_labels_300_10"
labels= pd.read_pickle(path)

In [19]:
labels = labels.values
values = values.values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(values,labels, test_size=0.2, random_state=42)

In [21]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [15]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (2735, 300, 1)
X_test shape: (684, 300, 1)
y_train shape: (2735, 1)
y_test shape: (684, 1)


In [22]:
nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

In [23]:
input_shape = X_train.shape[1:]

In [24]:
enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

In [25]:
input_shape

(300, 1)

In [26]:
(X_train.shape)

(2735, 300, 1)

In [ ]:
padding = 'valid'
input_layer = keras.layers.Input(input_shape)

conv1 = keras.layers.Conv1D(filters=6,kernel_size=7,padding=padding,activation='sigmoid')(input_layer)
conv1 = keras.layers.AveragePooling1D(pool_size=3)(conv1)

conv2 = keras.layers.Conv1D(filters=12,kernel_size=7,padding=padding,activation='sigmoid')(conv1)
conv2 = keras.layers.AveragePooling1D(pool_size=3)(conv2)

flatten_layer = keras.layers.Flatten()(conv2)

output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(flatten_layer)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(),
                metrics=['accuracy'])

# model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
#                 metrics=['accuracy'])


In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(nb_classes, activation='softmax'))

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
from tensorflow.keras.layers import Concatenate, Input, Conv1D, MaxPooling1D, LSTM, GlobalMaxPooling1D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model

input_layer = Input(shape=input_shape)
conv_layer1 = Conv1D(filters=32, kernel_size=3, activation='relu')(input_layer)
conv_layer2 = Conv1D(filters=32, kernel_size=3, activation='relu')(conv_layer1)
maxpool_layer = MaxPooling1D(pool_size=2)(conv_layer2)
rnn_layer = LSTM(units=64, return_sequences=True)(maxpool_layer)
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(maxpool_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Flatten()(cnn_layer)
rnn_layer = Flatten()(rnn_layer)
rnn_layer = Dense(64)(rnn_layer)
combined_layer = Concatenate()([rnn_layer, cnn_layer])
global_pooling_layer = GlobalMaxPooling1D()(combined_layer[..., None])  # Add an extra dimension
dense_layer = Dense(units=128, activation='relu')(global_pooling_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(units=nb_classes, activation='softmax')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, GlobalMaxPooling1D, concatenate

# Single input
input_layer = Input(shape=input_shape)

# Convolutional branch
conv_branch = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
conv_branch = MaxPooling1D(pool_size=2)(conv_branch)

# Recurrent branch
rnn_branch = LSTM(units=64, return_sequences=True)(input_layer)

# Apply global max pooling to both branches
conv_branch = GlobalMaxPooling1D()(conv_branch)
rnn_branch = GlobalMaxPooling1D()(rnn_branch)

# Concatenate the outputs of the convolutional and recurrent branches
concatenated = concatenate([conv_branch, rnn_branch])

# Classification layers
dense_layer = Dense(units=128, activation='relu')(concatenated)
output_layer = Dense(units=nb_classes, activation='softmax')(dense_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, GlobalMaxPooling1D, concatenate

# Single input
input_layer = Input(shape=input_shape)

# Convolutional branch
conv_branch = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
conv_branch = MaxPooling1D(pool_size=2)(conv_branch)

# First recurrent branch
rnn_branch1 = LSTM(units=64, return_sequences=True)(input_layer)

# Second recurrent branch
rnn_branch2 = LSTM(units=64, return_sequences=True)(input_layer)

# Apply global max pooling to the convolutional branch
conv_branch = GlobalMaxPooling1D()(conv_branch)

# Apply global max pooling to the recurrent branches
rnn_branch1 = GlobalMaxPooling1D()(rnn_branch1)
rnn_branch2 = GlobalMaxPooling1D()(rnn_branch2)

# Concatenate the outputs of all branches
concatenated = concatenate([conv_branch, rnn_branch1, rnn_branch2])

# Classification layers
dense_layer = Dense(units=128, activation='relu')(concatenated)
output_layer = Dense(units=nb_classes, activation='softmax')(dense_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [27]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, GlobalMaxPooling1D, concatenate

# Single input
input_layer = Input(shape=input_shape)

conv_branch1 = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)

# Convolutional branch
conv_branch = Conv1D(filters=64, kernel_size=3, activation='relu')(conv_branch1)
conv_branch = MaxPooling1D(pool_size=2)(conv_branch)

# First recurrent branch
rnn_branch1 = LSTM(units=64, return_sequences=True)(conv_branch1)

# Second recurrent branch
rnn_branch2 = LSTM(units=64, return_sequences=True)(input_layer)

# Apply global max pooling to the convolutional branch
conv_branch = GlobalMaxPooling1D()(conv_branch)

# Apply global max pooling to the recurrent branches
rnn_branch1 = GlobalMaxPooling1D()(rnn_branch1)
rnn_branch2 = GlobalMaxPooling1D()(rnn_branch2)

# Concatenate the outputs of all branches
concatenated = concatenate([conv_branch, rnn_branch1, rnn_branch2])

# Classification layers
dense_layer = Dense(units=128, activation='relu')(concatenated)
output_layer = Dense(units=nb_classes, activation='softmax')(dense_layer)

# Create the model
model1 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [28]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 298, 128)     512         ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 296, 64)      24640       ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 148, 64)      0           ['conv1d_1[0][0]']               
                                                                                              

In [38]:
!pip install pydot


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
from tensorflow.keras.utils import model_to_dot
import pydot
from IPython.display import Image
# Convert the model to a graph in DOT format
dot_graph = model_to_dot(model1).to_string()

# Create a pydot graph from the DOT string
graph = pydot.graph_from_dot_data(dot_graph)
graph[0].write_png('model.png')

# Display the image
Image(filename='model.png')

ImportError: You must install pydot (`pip install pydot`) for model_to_dot to work.

In [30]:
tf.keras.utils.plot_model(model1,'model.png', show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [41]:
from tensorflow.keras.callbacks import EarlyStopping

batch_size = 32
nb_epochs = 500

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Compile the model
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with optimizations
hist = model1.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs,
                 validation_data=(X_test, y_test), callbacks=[early_stopping])

model1.save('model_parallel_azad.hdf5')

Epoch 1/500
86/86 [==============================] - 19s 186ms/step - loss: 1.2075 - accuracy: 0.3828 - val_loss: 1.1371 - val_accuracy: 0.6301
Epoch 2/500
86/86 [==============================] - 15s 176ms/step - loss: 1.1361 - accuracy: 0.4117 - val_loss: 1.1146 - val_accuracy: 0.6287
Epoch 3/500
86/86 [==============================] - 15s 174ms/step - loss: 1.0970 - accuracy: 0.4673 - val_loss: 1.0575 - val_accuracy: 0.4298
Epoch 4/500
86/86 [==============================] - 15s 172ms/step - loss: 0.9777 - accuracy: 0.5722 - val_loss: 0.8210 - val_accuracy: 0.6082
Epoch 5/500
86/86 [==============================] - 15s 171ms/step - loss: 0.7932 - accuracy: 0.5879 - val_loss: 0.7022 - val_accuracy: 0.6038
Epoch 6/500
86/86 [==============================] - 16s 191ms/step - loss: 0.9474 - accuracy: 0.5441 - val_loss: 0.8503 - val_accuracy: 0.6301
Epoch 7/500
86/86 [==============================] - 17s 194ms/step - loss: 0.8160 - accuracy: 0.5828 - val_loss: 0.7210 - val_accuracy:

In [ ]:
batch_size = 64
nb_epochs = 750

hist = model1.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs, validation_data=(X_test, y_test))

model1.save('model_parallel3.hdf5')

In [ ]:
model1.save('model_parallel3.hdf5')

In [ ]:
test_values = pd.read_pickle('test_data_values')


In [53]:

test_labels = pd.read_pickle('test_labels_300_10')
test_labels.to_excel("test_label_300_10.xlsx")

In [50]:
test_values = pd.read_pickle('test_data_values_300_10')

In [52]:
test_values.to_excel("test_300_10.xlsx")

In [44]:
test_values = test_values.values

In [45]:
#test_values = test_values.values
test_values = test_values.reshape((test_values.shape[0], test_values.shape[1], 1))

In [46]:
test_values.shape

(2046, 300, 1)

In [47]:
predictions = model1.predict(test_values)


64/64 [==============================] - 4s 45ms/step


In [48]:
predictions

array([[7.1996611e-01, 2.7991813e-01, 2.8486701e-32, 3.9707193e-06,
        9.1516260e-05, 2.0316718e-05],
       [1.2887692e-03, 9.9853849e-01, 7.2795458e-36, 1.9557078e-07,
        1.7170074e-04, 8.2074331e-07],
       [2.8859754e-06, 9.9988651e-01, 0.0000000e+00, 6.7296742e-09,
        1.1062447e-04, 2.1289589e-08],
       ...,
       [3.4590322e-09, 2.1541282e-04, 2.3986621e-01, 4.6660103e-02,
        2.8359180e-08, 7.1325821e-01],
       [8.9666915e-14, 3.9594557e-08, 9.9522829e-01, 3.4261236e-04,
        9.1347320e-12, 4.4290787e-03],
       [1.0243467e-08, 3.8492185e-04, 3.8140662e-02, 2.6790526e-02,
        4.7776886e-08, 9.3468386e-01]], dtype=float32)

In [49]:
prediction_list = []

threshold = 0.0001

# Iterate over each row of predictions
for row in predictions:
    # Sort the probabilities in descending order and get the indices
    sorted_indices = np.argsort(row)[::-1]

    # Check if the highest prediction is above the threshold
    if row[sorted_indices[0]] >= threshold:
        prediction1 = sorted_indices[0]
    else:
        prediction1 = 3

    # Check if the second highest prediction is above the threshold
    if row[sorted_indices[1]] >= threshold:
        prediction2 = sorted_indices[1]
    else:
        prediction2 = 3

    # Add the predictions to the list
    prediction_list.append([prediction1, prediction2])

# Create the DataFrame from the list
#df = pd.DataFrame(prediction_list, columns=['prediction1', 'prediction2'])
prediction_list = np.array(prediction_list)
labels = ['Bias', 'Drift', 'Gain', 'NoFault', 'Outliers', 'Precisiondegradation']
label_encoder = LabelEncoder()
label_encoder.fit(labels)
predicted_labels_1 = label_encoder.inverse_transform(prediction_list[:,0])
predicted_labels_2 = label_encoder.inverse_transform(prediction_list[:,1])
df_predictions = pd.DataFrame({'Prediction': predicted_labels_1, 'Prediction2': predicted_labels_2})

# Display the resulting DataFrame
print(df_predictions)
df_predictions.to_excel('predictions4.xlsx')



                Prediction           Prediction2
0                     Bias                 Drift
1                    Drift                  Bias
2                    Drift              Outliers
3                    Drift               NoFault
4                    Drift               NoFault
...                    ...                   ...
2041  Precisiondegradation               NoFault
2042  Precisiondegradation                  Gain
2043  Precisiondegradation                  Gain
2044                  Gain  Precisiondegradation
2045  Precisiondegradation                  Gain

[2046 rows x 2 columns]


In [ ]:
test_values = 

In [ ]:
top_two_probs = np.sort(predictions, axis=1)[:, -2:]
top_two_probs

In [ ]:
top_two_indices = np.argsort(predictions, axis=1)[:, -2:]
predicted_label_indices = top_two_indices[:, ::-1]
first_predictions = predicted_label_indices[:,0]
second_predictions = predicted_label_indices[:,1]
labels = ['Bias', 'Drift', 'Gain', 'NoFault', 'Outliers', 'Precisiondegradation']
label_encoder = LabelEncoder()
label_encoder.fit(labels)
predicted_labels_1 = label_encoder.inverse_transform(first_predictions)
predicted_labels_2 = label_encoder.inverse_transform(second_predictions)
df_predictions = pd.DataFrame({'Prediction': predicted_labels_1, 'Prediction2': predicted_labels_2})

df_predictions


In [ ]:
df_predictions.to_excel('predictions.xlsx')

In [ ]:
labels = ['Bias', 'Drift', 'Gain', 'NoFault', 'Outliers', 'Precisiondegradation']
predicted_labels = np.argmax(predictions, axis=1)
predicted_label_indices = np.argmax(predictions, axis=1)

# top_two_indices = np.argsort(predictions, axis=1)[:, -2:]
# predicted_label_indices = top_two_indices[:, ::-1]

label_encoder = LabelEncoder()
label_encoder.fit(labels)

predicted_labels = label_encoder.inverse_transform(predicted_label_indices)

predicted_labels

In [ ]:
top_two_probs = np.sort(predictions, axis=1)[:, -2:]
threshold = 0.5  # Adjust this threshold according to your needs

predicted_labels = np.zeros((predictions.shape[0], 2), dtype=np.int32)
predicted_labels[:, 0] = predicted_label_indices

mask = top_two_probs[:, 1] > threshold
predicted_labels[mask, 1] = predicted_label_indices[mask]


In [ ]:
top_two_probs = np.sort(predictions, axis=1)[:, -2:]
threshold = 0.5  # Adjust this threshold according to your needs

predicted_labels = np.zeros((predictions.shape[0], 1), dtype=np.int32)
predicted_labels[:, 0] = predicted_label_indices[:predicted_labels.shape[0]]

mask = top_two_probs[:, 1] > threshold

predicted_labels_2 = np.zeros((predicted_labels.shape[0], 1), dtype=np.int32)
predicted_labels_2[mask] = predicted_label_indices[mask].reshape(-1, 1)

predicted_labels = np.concatenate((predicted_labels, predicted_labels_2), axis=1)


In [ ]:
prediction_column = label_encoder.inverse_transform(predicted_labels[:, 0])
prediction2_column = label_encoder.inverse_transform(predicted_labels[:, 1])

predictions_df = pd.DataFrame({'Prediction': prediction_column, 'Prediction2': prediction2_column})
print(predictions_df)



In [ ]:
predictions_df.to_excel('predictions.xlsx')

In [ ]:
predicted_label_indices

In [ ]:
predicted_labels_df = pd.DataFrame(predicted_labels)
predicted_labels_df.to_excel('predictions.xlsx')